## Dataset of the HMM

The main idea of the project consist in generating an Hidden Markov Model (HMM) with the Kunitz domain (BPTI). For it, we must presiously adquire a data set of protein sequences containing the Kunitz domain. So, we acessed PDB to extract the structures that have the following features:

* **Resolution** - We must choose structures with a low resolution, since in the opposite case it could compromise the reliability of our model. The threshold of 3 Å is chosen since it can model the interaction bonds and the distance between two carbons in the backbone. Then a resolution in the range of 0-3 Å has been chosen. Hydrogen bonds are crucial for the formation of secondary structure and protein-protein interactions. The lengths of hydrogen bonds depend on the polar residues that participate in the interaction which is why we use such a variable range. 

* **PFAM domain** - We include only proteins that match with the PFAM ID PF00014, associated to Kunitz/Bovine pancreatic trypsin inhibitor (BPTI) domain.

* **Length of polymer chain** - For this step it is critical to research the literature to be determine the usual length of this type of domain. We include only proteins with length between 50-70 aa.  We include just the domains themselves. Then, we avoid proteins that include the domain in their sequences. 

**OUR QUERY**: Refinement Resolution = [ 0 - 3 ] AND ( Identifier = "PF00014" AND Annotation Type = "Pfam" ) AND Total Number of Polymer Residues per Deposited Model = [ 50 - 70 ]

After setting the options indicated above in the advanced search of the PDB website, we obtained **28 structures**. We must download the list of proteins setting as Tabular Report Sequence type.

The command below create a FASTA format (the protein ID, the chain and sequence) with the structures in `kunitz.csv` file. Moreover, we refiltered all the structures composed by a number of residues between 50 and 70. After this filtering process, we got **26 protein sequences** in the FASTA file.

In [60]:
%%bash
sed -i 's/\, / /g' kunitz.csv 
sed -i 's/\"//g' kunitz.csv
awk -F ',' '{if ($7 <= 70 && $7 >= 50) print ">"$1":"$3 "\n" $6}' kunitz.csv > kunitz_pdb.fasta
grep ">" kunitz_pdb.fasta | wc -l

26


####  Clustering the sequences

In [ ]:
# ~/blast-2.2.26/bin/blastclust -i kunitz_pdb.fasta -o kunitz.clust -S 99 -L 0.99

Blastclust command allows us to cluster sequences according to the sequence identity (-S) and the lenght of the coverage (-L). Where we chose high values to sequence identity (99) and to cpverage (0.99) because we wanted the sequences of each cluster to be as similar as possible.

We got 15 clusters but in one of them contains proteins of the APP not the BPTI domain (second line in `kunitz.clust` file).

View the clustering result, in the file `kunitz.clust`:

In [64]:
! cat kunitz.clust

1BPI:A 4PTI:A 5PTI:A 6PTI:A 9PTI:A 
2FJZ:A 2FK1:A 2FK2:A 2FMA:A 
1G6X:A 1K6U:A 1QLQ:A 
1KNT:A 1KTH:A 2KNT:A 
6Q61:A 
3OFW:A 
5YV7:A 
1DTX:A 
1BPT:A 
1BTI:A 
1FAN:A 
1NAG:A 
7PTI:A 
8PTI:A 
5M4V:A 


#### Choosing the best structure of each cluster

In this point we have to choose the **best structure** of each cluster. Then, we have to check which of the structures has the lowest, in this case is the best, resolution. We downloaded from PDB the csv that includes the [resolution](https://www.rcsb.org/search?request=%7B%22query%22%3A%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22range_closed%22%2C%22negation%22%3Afalse%2C%22value%22%3A%5B0%2C3%5D%2C%22attribute%22%3A%22rcsb_entry_info.resolution_combined%22%7D%2C%22node_id%22%3A0%7D%5D%7D%2C%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_polymer_entity_annotation.annotation_id%22%2C%22negation%22%3Afalse%2C%22operator%22%3A%22exact_match%22%2C%22value%22%3A%22PF00014%22%7D%2C%22node_id%22%3A1%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_polymer_entity_annotation.type%22%2C%22operator%22%3A%22exact_match%22%2C%22value%22%3A%22Pfam%22%7D%2C%22node_id%22%3A2%7D%5D%2C%22label%22%3A%22nested-attribute%22%7D%5D%7D%2C%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22range_closed%22%2C%22negation%22%3Afalse%2C%22value%22%3A%5B50%2C70%5D%2C%22attribute%22%3A%22rcsb_entry_info.deposited_polymer_monomer_count%22%7D%2C%22node_id%22%3A3%7D%5D%7D%5D%2C%22label%22%3A%22text%22%7D%5D%2C%22label%22%3A%22query-builder%22%7D%2C%22return_type%22%3A%22entry%22%2C%22request_options%22%3A%7B%22pager%22%3A%7B%22start%22%3A0%2C%22rows%22%3A100%7D%2C%22scoring_strategy%22%3A%22combined%22%2C%22sort%22%3A%5B%7B%22sort_by%22%3A%22score%22%2C%22direction%22%3A%22desc%22%7D%5D%7D%2C%22request_info%22%3A%7B%22src%22%3A%22ui%22%2C%22query_id%22%3A%220fb8cbb58f80d696e95857c7598870d5%22%7D%7D) and write a script `best_structure.py` to filter for the sequence with the best resolution of **each cluster**.

In [6]:
! python best_structure.py structures_resolution.csv final_kunitz_clusters.clust > ids_kunitz.txt

The file `ids_kunitz.txt` contains a list with the best structures to each cluster.

In [7]:
! cat ids_kunitz.txt

5PTI:A
2FMA:A
1G6X:A
1KTH:A
6Q61:A
3OFW:A
5YV7:A
1DTX:A
1BPT:A
1BTI:A
1FAN:A
1NAG:A
7PTI:A
8PTI:A
5M4V:A


#### PDBe Fold: Multiple Structure Alignment (MSA)

Once we found the all the **best structures** of each cluster. We use the file `ids_kunitz.txt` to do a multiple structure alignment in [PDBe fold](https://www.ebi.ac.uk/msd-srv/ssm/).

We obtained a several results. We can highlight three matrices: RMSD, Q-score and Sequence identity.

Cross-structure statistics - **RMSD**:

![title](img/PDBe-RMSD.png)

All the values are good enough, down to 3 Å. We can highlight that both the second row and column, which corresponds to the 2FMA structure, are the highest values.

**Q-score**:

![title](img/PDBe-Qscore.png)

As we well know, the higher the Q-score, the more similarity there will be. It also depends on the size of the structure that we align. In our case, it is a short domain.

The worst values are in the second column and row as well (2FMA).

**Sequence identity**:

![title](img/PDBe-Sequence.png)

When we examine the sequence identity, we also realise that something is wrong with the 2FMA structure. The percentages is between 0 to 0.15. Which makes us think that it does not belong to the family of proteins with which it is being compared.

After the results obtained previously, we have verified that the 2FMA structure and the cluster belongs to the Alzheimer's Amyloid Precursor Protein (APP) but not to the Bovine Pancreatic Trypsin Inhibitor (BPTI).

Since the entire cluster (2FMA, 2FJZ, 2FK1 and 2FK2) is responsible for the misalignment since it does not belong to the domain we want, it was decided to remove it.

We have to change the list of best clusters and remove 2FMA, 2FJZ, 2FK1 and 2FK2 and rerun the PDB fold with the updated list in the file `final_kunitz_clusters.txt`.

In [71]:
! cat final_ids_kunitz.txt 

5PTI:A 
1G6X:A 
1KTH:A 
6Q61:A 
3OFW:A 
5YV7:A 
1DTX:A 
1BPT:A 
1BTI:A 
1FAN:A 
1NAG:A 
7PTI:A 
8PTI:A 
5M4V:A 


We repeated the MSA in [PDBe fold](https://www.ebi.ac.uk/msd-srv/ssm/).

Cross-structure statistics - **RMSD**:

![title](img/PDBe-RMSD-final.png)

**Q-score**:

![title](img/PDBe-Qscore-final.png)

**Sequence identity**:

![title](img/PDBe-Sequence-final.png)

In [86]:
! cat final_all_aln_kunitz_clusters.fasta.seq

>PDB:5pti:A STRUCTURE OF BOVINE PANCREATIC TRYPSIN INHIBITOR. 
--rPDFCLEPP-YtGPCKARIIRYFYNAKAGLCQTFVYGGCrA-KRNNFKSAEDCMRTCGga

>PDB:1g6x:A ULTRA HIGH RESOLUTION STRUCTURE OF BOVINE PANCREAT
--rPDFCLEPP-YaGACRARIIRYFYNAKAGLCQTFVYGGCrA-KRNNFKSAEDCLRTCGga

>PDB:1kth:A THE ANISOTROPIC REFINEMENT OF KUNITZ TYPE DOMAIN C
--eTDICKLPK-DeGTCRDFILKWYYDPNTKSCARFWYGGCgG-NENKFGSQKECEKVCApv

>PDB:6q61:A PORE-MODULATING TOXINS EXPLOIT INHERENT SLOW INACT
-drPSLCDLPA-DsGSGTKAEKRIYYNSARKQCLRFDYTGQgG-NENNFRRTYDCARTCLyt

>PDB:3ofw:A CRYSTAL STRUCTURE OF RECOMBINANT KUNITZ TYPE SERIN
--eASICSEPK-KvGRCKGYFPRFYFDSETGKCTPFIYGGCgG-NGNNFETLHQCRAICR--

>PDB:5yv7:A RACEMIC X-RAY STRUCTURE OF CALCICLUDINE
wqpPWYCKEPV-RiGSCKKQFSSFYFKWTAKKCLPFLFSGCgG-NANRFQTIGECRKKCLgk

>PDB:1dtx:A CRYSTAL STRUCTURE OF ALPHA-DENDROTOXIN FROM THE GR
eprRKLCILHR-NpGRCYDKIPAFYYNQKKKQCERFDWSGCgG-NSNRFKTIEECRRTCIg-

>PDB:1bpt:A CREVICE-FORMING MUTANTS OF BPTI: CRYSTAL STRUCTURE
--rPDFCLEPP-YtGPCKARIIRYFANAKAGLCQTFVYGGCrA-KRNNFKSAEDCMRTC

The summary of the MSA result is in `MSA_Summary.txt` and the results of PDBe is in `PDBe-results.txt`. 

## HMM Building

#### Preparing data clusters to HMM input

We generated a copy from `final_all_aln_kunitz_clusters.fasta.seq` in `bpti_only.fasta` to modify data into capital letters. 

In [93]:
! cp final_all_aln_kunitz_clusters.fasta.seq bpti_only.fasta 
! awk '{print toupper($1)}' bpti_only.fasta > input_HMM.fasta
! cat input_HMM.fasta

>PDB:5PTI:A
--RPDFCLEPP-YTGPCKARIIRYFYNAKAGLCQTFVYGGCRA-KRNNFKSAEDCMRTCGGA

>PDB:1G6X:A
--RPDFCLEPP-YAGACRARIIRYFYNAKAGLCQTFVYGGCRA-KRNNFKSAEDCLRTCGGA

>PDB:1KTH:A
--ETDICKLPK-DEGTCRDFILKWYYDPNTKSCARFWYGGCGG-NENKFGSQKECEKVCAPV

>PDB:6Q61:A
-DRPSLCDLPA-DSGSGTKAEKRIYYNSARKQCLRFDYTGQGG-NENNFRRTYDCARTCLYT

>PDB:3OFW:A
--EASICSEPK-KVGRCKGYFPRFYFDSETGKCTPFIYGGCGG-NGNNFETLHQCRAICR--

>PDB:5YV7:A
WQPPWYCKEPV-RIGSCKKQFSSFYFKWTAKKCLPFLFSGCGG-NANRFQTIGECRKKCLGK

>PDB:1DTX:A
EPRRKLCILHR-NPGRCYDKIPAFYYNQKKKQCERFDWSGCGG-NSNRFKTIEECRRTCIG-

>PDB:1BPT:A
--RPDFCLEPP-YTGPCKARIIRYFANAKAGLCQTFVYGGCRA-KRNNFKSAEDCMRTCG--

>PDB:1BTI:A
--RPDFCLEPP-YTGPCKARIIRYAYNAKAGLCQTFVYGGCRA-KRNNFKSAEDCMRTCGGA

>PDB:1FAN:A
--RPDFCLEPP-YTGPCKARIIRYFYNAKAGLCQTFVYGGCRA-KRNNAKSAEDCMRTCGGA

>PDB:1NAG:A
--RPDFCLEPP-YTGPCKARIIRYFYNAKAGLCQTFVYGGCRA-KRGNFKSAEDCMRTCG--

>PDB:7PTI:A
--RPDFCLEPP-YTGPCKARIIRYFYNAKAGLAQTFVYGGCRA-KRNNFKSAEDAMRTCGGA

>PDB:8PTI:A
--RPDFCLEPPYT-GPCKARIIRYFYNAKAGLCQTFVGGGC-RAKRNNFKSAEDCMRTCGGA

>PDB:5M4V:A


#### Building a Profile HMM Based on Structural Alignment

Using the HMMER package we build the HMM coming from the MSA with the following command:

`hmmbuild [outfile] [infile]` - Build a profile HMM from an input multiple alignment.

In [1]:
# hmmbuild bpti_kunitz.hmm input_HMM.fasta

#### Visualizing the model

In this file `bpti_kunitz.hmm` we could check the profile HMM (statistical model of multiple sequences alignment, in this case 14 sequences). It captures position-specific information about how conserved each column of the alignment is, and which residues are likely.

We can see that the model has 58 matches, it is the length of our alignment, and also we can see all the information about the positions of the transitons (state transition probabilities).

In [9]:
! head -30 bpti_kunitz.hmm

HMMER3/f [3.1b2 | February 2015]
NAME  input_HMM
LENG  58
ALPH  amino
RF    no
MM    no
CONS  yes
CS    no
MAP   yes
DATE  Tue Dec 22 21:19:33 2020
NSEQ  14
EFFN  1.760254
CKSUM 3868968390
STATS LOCAL MSV       -8.8288  0.71895
STATS LOCAL VITERBI   -9.1132  0.71895
STATS LOCAL FORWARD   -3.8349  0.71895
HMM          A        C        D        E        F        G        H        I        K        L        M        N        P        Q        R        S        T        V        W        Y   
            m->m     m->i     m->d     i->m     i->i     d->m     d->d
  COMPO   2.63348  2.65678  3.16597  2.76129  2.89267  2.52955  3.92526  3.29068  2.51956  2.96800  4.09400  2.90837  3.30591  3.15023  2.72848  2.78161  2.89476  3.21779  4.75114  3.06560
          2.68628  4.42235  2.77494  2.73104  3.46364  2.40523  3.72505  3.29364  2.67751  2.69365  4.24700  2.90357  2.73720  3.18106  2.89811  2.37897  2.77530  2.98528  4.58284  3.61513
          0.29448  1.38679  5.25883  0.52156  0.90041  0

The parameters are all stored as negative natural log probabilities and in the cases of a value is 0 is stored as '*'.

**HMM** - Flags the start of the main model section.

**COMPO** - The model’s overall average match state emission probabilities, which are used as a background residue composition in the “filter null” model.

Example:

**Match emission line** -  1   3.00537  5.33421  3.12865  1.97142  4.72620  3.66387  3.82224  4.14924  2.36701  3.62868  4.43611  3.16732  2.91009  2.94677  1.27140  2.97941  3.22562  3.76912  5.73562  4.42894      3 r - - -

**Insert emission line** -  2.68618  4.42225  2.77519  2.73123  3.46354  2.40513  3.72494  3.29354  2.67741  2.69355  4.24690  2.90347  2.73739  3.18146  2.89801  2.37887  2.77519  2.98518  4.58477  3.61503

**State transition line** -  0.01604  4.53648  5.25883  0.61958  0.77255  0.48576  0.95510
(m - match, i - insertion, d - delection)

#### Sequence LOGO after profile HMM 

In [skyling.org](http://skylign.org/) web page we uploaded our `bpti_kunitz.hmm` file to generate the sequence logo.

**A static image of the logo**
![title](img/hmmlogo.png)

**A static image of the logo scaled to the theoretical maximum height**
![title](img/hmmlogo_maximum_height.png)

It can be seen from the sequence logo that the 6 cysteines that are important to keep the structure compact are there. And other sites that are potentially important for protein function, such as tyrosines (Y).

## Getting data to test the HMM

#### Extracting the positive set

We have to create the positive and negative datasets. The positive dataset **includes ALL proteins** that contain Kunitz domains within their structures, while the **negative** dataset is composed of proteins that do **NOT include** it from UniProt. 

We searched in Uniprot advanced these following properties:
    
* Cross-reference > Family and domain databases > Pfam (PF00014)
* Reviewed > Reviewed
    
We retrieved **360** entries that contain the kunitz domain.

In [47]:
%%bash
grep '>' all_kunitz.fasta | wc -l 
# head all_kunitz.fasta

360


#### Extracting the negative set

The **negative set** was retrieved from UniProt advance search:
  * NOT Cross-references -> Family and domain databases -> Pfam (PF00014)
  * Reviewed -> Reviewed

The search returned **563,612** entries:

In [48]:
%%bash
grep '>' Not_kunitz.fasta | wc -l
# head Not_kunitz.fasta

563612


#### Editing the files format of positive and negative set

We edited the header of the positive and negative sets that we retrieved from Uniprot. The idea is only get the ID of Uniprot from header and the sequence.

In [54]:
%%bash

awk '{if (substr($0, 1, 1) == ">"){split($0, a, "|"); print ">" a[2]} else {print $0}}' all_kunitz.fasta > positive_set.fasta
awk '{if (substr($0, 1, 1) == ">"){split($0, a, "|"); print ">" a[2]} else {print $0}}' Not_kunitz.fasta > negative_set.fasta

echo 'Positive set:'
head positive_set.fasta
echo ''
echo 'Number of positive set'
grep ">" positive_set.fasta | wc -l
echo ''
echo 'Negative set:'
head negative_set.fasta
echo ''
echo 'Number of negative set'
grep ">" negative_set.fasta | wc -l

Positive set:
>P10646
MIYTMKKVHALWASVCLLLNLAPAPLNADSEEDEEHTIITDTELPPLKLMHSFCAFKADD
GPCKAIMKRFFFNIFTRQCEEFIYGGCEGNQNRFESLEECKKMCTRDNANRIIKTTLQQE
KPDFCFLEEDPGICRGYITRYFYNNQTKQCERFKYGGCLGNMNNFETLEECKNICEDGPN
GFQVDNYGTQLNAVNNSLTPQSTKVPSLFEFHGPSWCLTPADRGLCRANENRFYYNSVIG
KCRPFKYSGCGGNENNFTSKQECLRACKKGFIQRISKGGLIKTKRKRKKQRVKIAYEEIF
VKNM
>Q7TQN3
MCAPGYHRFWFHWGLLLLLLLEAPLRGLALPPIRYSHAGICPNDMNPNLWVDAQSTCKRE
CETDQECETYEKCCPNVCGTKSCVAARYMDVKGKKGPVGMPKEATCDHFMCLQQGSECDI

Number of positive set
360

Negative set:
>Q4R4R9
MASATDSRYGQKESSDQNFDYMFKILIIGNSSVGKTSFLFRYADDSFTPAFVSTVGIDFK
VKTIYRNDKRIKLQIWDTAGQERYRTITTAYYRGAMGFILMYDITNEESFNAVQDWSTQI
KTYSWDNAQVLLVGNKCDMEDERVVSSERGRQLADHLGFEFFEASAKDNINVKQTFERLV
DVICEKMSESLDTADPAVTGAKQGPQLSDQQVPPHQDCAC
>P0C6Y3
MASSLKQGVSPKLRDVILVSKDIPEQLCDALFFYTSHNPKDYADAFAVRQKFDRNLQTGK
QFKFETVCGLFLLKGVDKITPGVPAKVLKATSKLADLEDIFGVSPFARKYRELLKTACQW
SLTVETLDARAQTLDEIFDPTEILWLQVAAKIQVSAMAMRRLVGEVTAKVMDALGSNMSA
LFQIFKQQIVRIFQKALAIFENVSELPQRIAALKMAFAKCAKSITVVVMERTLVVREFAG

Number of 

#### BLAST search
(to find proteins already used HMM building)

We must eliminate the sequence to be used for the MSA because they would skew the results. So, we transformed the positive set into a database in order to blast the sequences from MSA to thems.

In [ ]:
# formatdb -i positive_set.fasta -p T

Then we run BLAST of the 360 sequences, `positive_set.fasta`, with of the pdb and remove all sequences that have and E-value that is higher than a given threshold (0.001).

In [ ]:
# blastpgp -i input_HMM.fasta -d positive_set.fasta -m 8 -e 0.001 -o positive_set.bl8

And for identifying the IDs we must select only proteins that have percentage of identity higher than 99% (because we want exact matches). The command for retrieving these IDs into a text file is the following:

In [10]:
%%bash
awk '{if ($3 >=99) print $2}' positive_set.bl8 | sort -u > ids_to_eliminate.txt

Warning: Keeping the sequences to be used for the MSA would skew the results. We must eliminate the sequence to be used for the MSA because them. So, we transformed the positive set into a database in order to blast the sequences from MSA to thems.

And in order to eliminate the corresponding header + sequence from the positive set we run the following Python script `negative_extract_seq.py`:

In [45]:
%%bash
grep ">" positive_set.fasta | wc -l
python negative_extract_seq.py ids_to_eliminate.txt positive_set.fasta > clean_positive_set.fasta
grep ">" clean_positive_set.fasta | wc -l

360
356


## Preparing data for 2-fold Cross-validation

#### Splitting the positive dataset

Now we split our positive dataset into two parts after having randomised the protein IDs. Then we check that there are not overlaps with the last command.

In [62]:
%%bash
grep ">" clean_positive_set.fasta | sed 's/>//' | sort -R > random_positive.txt
head -n 178 random_positive.txt > set_r1_positive.txt
tail -n +179 random_positive.txt > set_r2_positive.txt

In [63]:
! wc -l set_r1_positive.txt

178 set_r1_positive.txt


In [64]:
! wc -l set_r2_positive.txt

178 set_r2_positive.txt


#### Creating fasta file to set_r1_positive and set_r2_positive

The dataset is not too large, so we can use the same (slow) script above to extract the sequences from the positive set using the list of IDs.

So only the protein in the list is selected but the other half is left.

In [67]:
%%bash
python extract_seq.py set_r1_positive.txt clean_positive_set.fasta > set_r1_positive.fasta
python extract_seq.py set_r2_positive.txt clean_positive_set.fasta > set_r2_positive.fasta

head set_r1_positive.fasta
grep '>' set_r1_positive.fasta | wc -l
echo ''
head set_r2_positive.fasta
grep '>' set_r2_positive.fasta | wc -l

>Q7TQN3
MCAPGYHRFWFHWGLLLLLLLEAPLRGLALPPIRYSHAGICPNDMNPNLWVDAQSTCKRE
CETDQECETYEKCCPNVCGTKSCVAARYMDVKGKKGPVGMPKEATCDHFMCLQQGSECDI
WDGQPVCKCKDRCEKEPSFTCASDGLTYYNRCFMDAEACSKGITLSVVTCRYHFTWPNTS
PPPPETTVHPTTASPETLGLDMAAPALLNHPVHQSVTVGETVSFLCDVVGRPRPELTWEK
QLEDRENVVMRPNHVRGNVVVTNIAQLVIYNVQPQDAGIYTCTARNVAGVLRADFPLSVV
RGGQARATSESSLNGTAFPATECLKPPDSEDCGEEQTRWHFDAQANNCLTFTFGHCHHNL
NHFETYEACMLACMSGPLAICSLPALQGPCKAYVPRWAYNSQTGLCQSFVYGGCEGNGNN
FESREACEESCPFPRGNQHCRACKPRQKLVTSFCRSDFVILGRVSELTEEQDSGRALVTV
DEVLKDEKMGLKFLGREPLEVTLLHVDWTCPCPNVTVGETPLIIMGEVDGGMAMLRPDSF
178

>P10646
MIYTMKKVHALWASVCLLLNLAPAPLNADSEEDEEHTIITDTELPPLKLMHSFCAFKADD
GPCKAIMKRFFFNIFTRQCEEFIYGGCEGNQNRFESLEECKKMCTRDNANRIIKTTLQQE
KPDFCFLEEDPGICRGYITRYFYNNQTKQCERFKYGGCLGNMNNFETLEECKNICEDGPN
GFQVDNYGTQLNAVNNSLTPQSTKVPSLFEFHGPSWCLTPADRGLCRANENRFYYNSVIG
KCRPFKYSGCGGNENNFTSKQECLRACKKGFIQRISKGGLIKTKRKRKKQRVKIAYEEIF
VKNM
>P00981
SGHLLLLLGLLTLWAELTPVSGAAKYCKLPLRIGPCKRKIPSFYYKWKAKQCLPFDYSGC
GGNANRFKTIEECRRTCVG
178


#### Splitting the negative set

We generate a file containing **only the protein ID** of all 563612 **negative** Swissprot entries: using a pipeline that takes , is used to The negative dataset needs less analysis. From the FASTA file containing all the 563612 entries the headers were reduced to the protein ID using the first command below. Then we extracted the protein IDs and we sorted them randomly. We splited the list of IDs in two half.

In [66]:
%%bash
head negative_set.fasta
echo ''
grep ">" negative_set.fasta | wc -l

>Q4R4R9
MASATDSRYGQKESSDQNFDYMFKILIIGNSSVGKTSFLFRYADDSFTPAFVSTVGIDFK
VKTIYRNDKRIKLQIWDTAGQERYRTITTAYYRGAMGFILMYDITNEESFNAVQDWSTQI
KTYSWDNAQVLLVGNKCDMEDERVVSSERGRQLADHLGFEFFEASAKDNINVKQTFERLV
DVICEKMSESLDTADPAVTGAKQGPQLSDQQVPPHQDCAC
>P0C6Y3
MASSLKQGVSPKLRDVILVSKDIPEQLCDALFFYTSHNPKDYADAFAVRQKFDRNLQTGK
QFKFETVCGLFLLKGVDKITPGVPAKVLKATSKLADLEDIFGVSPFARKYRELLKTACQW
SLTVETLDARAQTLDEIFDPTEILWLQVAAKIQVSAMAMRRLVGEVTAKVMDALGSNMSA
LFQIFKQQIVRIFQKALAIFENVSELPQRIAALKMAFAKCAKSITVVVMERTLVVREFAG

563612


Confirming that the fasta contains **only the ID and the sequence** 563612/2 = 281806 so we need to generate two files each containing **281806** sequences. We can do this the same way as before with the positives. First we generate a **randomly** sorted .txt containing the seq ID **only** and then we split the .txt into 2 files.

In [2]:
%%bash
grep ">" negative_set.fasta | sed 's/>//' | sort -R > random_negative.txt
head -n 281806 random_negative.txt > set_r1_negative.txt 
tail -n +281807 random_negative.txt > set_r2_negative.txt

wc -l set_r1_negative.txt
wc -l set_r2_negative.txt

281806 set_r1_negative.txt
281806 set_r2_negative.txt


Extracting sequences from the protein IDs of such a large file would take too much time with the script we used for the positive ones. A better idea is to download pyfasta. After having installed pyfasta we can use these commands to extract the sequences:

In [5]:
%%bash
# pip install pyfasta
# pyfasta extract --header --fasta negative_set.fasta --file set_r1_negative.txt > set_r1_negative.fasta
# pyfasta extract --header --fasta negative_set.fasta --file setr2_negative.txt > set_r2_negative.fasta

# OR

# python extract_seq.py set_r1_negative.txt negative_set.fasta > set_r1_negative.fasta
# python extract_seq.py set_r2_negative.txt negative_set.fasta > set_r2_negative.fasta

In [9]:
%%bash
head set_r1_negative.fasta
grep ">" set_r1_negative.fasta | wc -l 

echo '' 

head set_r2_negative.fasta
grep '>' set_r2_negative.fasta | wc -l 

>Q4R4R9
MASATDSRYGQKESSDQNFDYMFKILIIGNSSVGKTSFLFRYADDSFTPAFVSTVGIDFK
VKTIYRNDKRIKLQIWDTAGQERYRTITTAYYRGAMGFILMYDITNEESFNAVQDWSTQI
KTYSWDNAQVLLVGNKCDMEDERVVSSERGRQLADHLGFEFFEASAKDNINVKQTFERLV
DVICEKMSESLDTADPAVTGAKQGPQLSDQQVPPHQDCAC
>P0C6Y3
MASSLKQGVSPKLRDVILVSKDIPEQLCDALFFYTSHNPKDYADAFAVRQKFDRNLQTGK
QFKFETVCGLFLLKGVDKITPGVPAKVLKATSKLADLEDIFGVSPFARKYRELLKTACQW
SLTVETLDARAQTLDEIFDPTEILWLQVAAKIQVSAMAMRRLVGEVTAKVMDALGSNMSA
LFQIFKQQIVRIFQKALAIFENVSELPQRIAALKMAFAKCAKSITVVVMERTLVVREFAG
281806

>B2RQC6
MAALVLEDGSVLQGRPFGAAVSTAGEVVFQTGMVGYPEALTDPSYKAQILVLTYPLIGNY
GIPSDEEDEFGLSKWFESSEIHVAGLVVGECCPTPSHWSANCTLHEWLQQRGIPGLQGVD
TRELTKKLREQGSLLGKLVQKGTEPSALPFVDPNARPLAPEVSIKTPRVFNAGGAPRICA
LDCGLKYNQIRCLCQLGAEVTVVPWDHELDSQKYDGLFLSNGPGDPASYPGVVSTLSRVL
SEPNPRPVFGICLGHQLLALAIGAKTYKMRYGNRGHNQPCLLVGTGRCFLTSQNHGFAVD
ADSLPAGWAPLFTNANDCSNEGIVHDSLPFFSVQFHPEHRAGPSDMELLFDVFLETVREA
AAGNIGGQTVRERLAQRLCPPELPIPGSGLPPPRKVLILGSGGLSIGQAGEFDYSGSQAI
KALKEENIQTLLINPNIATVQTSQGLADKVYFLPITLHYVTQVIRNERPDGVLLTFGGQT
ALNCGVELTKAG

The command takes the FASTA file as input and another file containing the list of identifiers and it returns another file with the extracted sequences.

## Performance testing by 2-fold Cross-validation

#### HMM search - Search a protein profile HMM against proteins sequence database.

It takes as input an hmmfile and a FASTA sequence input.

Search the profile `bpti-kunitz.hmm` against protein sequences database `set_r1_positive.fasta`, `set_r2_positive.fasta`, `set_r1_negative.fasta` or `set_r2_negative.fasta`.   

To facilitate the file parsing we used the following options:

* -Z this argument produces E-values that are comparable among different runs because it makes them independent from the dataset size.
* --tblout : tabular output file
* --noali : omit the alignment section from the input. We might reduce the output volume, because we only want the E-value for classification
* --max : Turn all heuristic filters off (less speed, more power)

In [ ]:
hmmsearch -h | cat

##### First positive training set:

In [ ]:
# hmmsearch -Z 1 --noali --max --tblout training_r1_positive.hits bpti-kunitz.hmm set_r1_positive.fasta

##### First positive testing set:

In [ ]:
# hmmsearch -Z 1 --noali --max --tblout testing_r2_positive.hits bpti-kunitz.hmm set_r2_positive.fasta

##### Second negative training set:

In [ ]:
# hmmsearch -Z 1 --noali --max --tblout training_r1_negative.hits bpti-kunitz.hmm set_r1_negative.fasta

##### Second negative testing set:

In [26]:
# hmmsearch -Z 1 --noali --max --tblout testing_r2_negative.hits bpti-kunitz.hmm set_r2_negative.fasta

#### Generating one file that contains only the e-value, the name of the sequence and the type of the sequence:

This serves to find the optimal threshold to be able to distinguish between negatives (0) and positives (1) as classified by PFAM/Uniprot. So we extract:

* the protein ID \$1,
* E-value (of the single domain) from \$8

And we add the lable 1 (pos) or 0 (neg) to the corresponding entries.

**Positives training and testing set**:

In [4]:
%%bash
grep -v "^#" training_r1_positive.hits | awk '{print $1, $8, 1}'> training_r1_positive.out
grep -v "^#" testing_r2_positive.hits | awk '{print $1, $8, 1}'> testing_r2_positive.out

echo "The positives r1 set:"
head -2 training_r1_positive.out
wc -l training_r1_positive.out

echo ''
echo "The positives r2 set:"
head -2 testing_r2_positive.out
wc -l testing_r2_positive.out

The positives r1 set:
Q868Z9 5e-21 1
O54819 3.2e-25 1
178 training_r1_positive.out

The positives r2 set:
O76840 2.8e-22 1
Q02445 9.8e-26 1
178 testing_r2_positive.out


**Negatives training and testing set**:

In [3]:
%%bash
grep -v "^#" training_r1_negative.hits | awk '{print $1,$8,0}'> training_r1_negative.out
grep -v "^#" testing_r2_negative.hits | awk '{print $1,$8,0}'> testing_r2_negative.out

echo 'The negatives r1 set:'
head -2 training_r1_negative.out
wc -l training_r1_negative.out

echo ''
echo 'The negatives r2 set:'
head -2 testing_r2_negative.out
wc -l testing_r2_negative.out

The negatives r1 set:
P84555 7.8e-08 0
P84556 7.1e-07 0
136061 training_r1_negative.out

The negatives r2 set:
P0DJ63 1.5e-08 0
P83605 4.9e-08 0
136445 testing_r2_negative.out


#### Recovering IDs with an E-value above 10

Each file contains however only elements that had an e-value <= 10. Now we need to generate a file with the same layout.

This **obviously** does not need to be done with the positive set as none of the evalues returned are higher than 10 ;-)

In [7]:
%%bash
comm <(grep "^>" set_r2_negative.fasta | sed 's/>//' | sort) < (awk '{print $1}' testing_r2_negative.out | sort) | awk -F '\t' '{if ($1 != "") print $1, 10, 0}' > missing_negative_r2.out
comm <(grep "^>" set_r1_negative.fasta | sed 's/>//' | sort) < (awk '{print $1}' training_r1_negative.out | sort) | awk -F '\t' '{if ($1 != "") print $1, 10, 0}' > missing_negative_r1.out

echo 'The missing negatives r1 set:'
cat missing_negative_r1.out | wc -l
echo 'The missing negatives r2 set:'
cat missing_negative_r2.out | wc -l

The missing negatives r1 set:
145745
The missing negatives r2 set:
145361


#### Checking weather everything sums up and makes sense:

Remember the total amount of negative sequences downloaded was 563612!

In [9]:
%%bash
echo "Negative Set 1"
wc -l missing_negative_r1.out
wc -l training_r1_negative.out

echo ''
echo "Negative Set 2"
wc -l missing_negative_r2.out
wc -l testing_r2_negative.out

Negative Set 1
145745 missing_negative_r1.out
136061 training_r1_negative.out

Negative Set 2
145361 missing_negative_r2.out
136445 testing_r2_negative.out


In [8]:
%%bash
echo 'Sum negative set 1:'
expr 145745 + 136061 

echo ''
echo 'Sum negative set 2:'
expr 145361 + 136445

echo ''
echo 'Total negatives'
expr 281806 + 281806

Sum negative set 1:
281806

Sum negative set 2:
281806

Total negatives
563612


#### The sum checks out every thing should be alright

In [12]:
%%bash
cat missing_negative_r1.out training_r1_negative.out > both_r1.out
cat missing_negative_r2.out testing_r2_negative.out > both_r2.out

echo ''
echo "Missing and Training negative set r1:"
wc -l both_r1.out
head -3 both_r1.out

echo ''
echo "Missing and Testing negative set r2:"
wc -l both_r2.out
head -3 both_r2.out


Missing and Training negative set r1:
281806 both_r1.out
A0A023GPI8 10 0
A0A023PXL1 10 0
A0A023PXL7 10 0

Missing and Testing negative set r2:
281806 both_r2.out
A0A009IHW8 10 0
A0A023I4C8 10 0
A0A023I4D6 10 0


The py script below will be used to test the performance.

In [27]:
! cat both_r1.out training_r1_positive.out | wc -l

281984


In [1]:
%%bash
(cat both_r1.out training_r1_positive.out) > performance_r1.out 
python performance2.py performance_r1.out

P84555 FP
P84556 FP
P36235 FP
P71089 FP
Q91736 FP
Q91571 FP
Q91694 FP
Q91845 FP
Q9PS05 FP
Threshold: 1 
ACC: 0.7228566159782115 
Matthews (MCC): 0.04052518157303454 
TPR: 1.0 
FPR: 0.2773184389260697 
Positive pred val: 0.0022724951486058627 
The Matrix (CM): [[178.0, 78150.0], [0.0, 203656.0]] 

P84555 FP
P84556 FP
P36235 FP
P71089 FP
Q91736 FP
Q91571 FP
Q91694 FP
Q91845 FP
Q9PS05 FP
Threshold: 0.1 
ACC: 0.9634837437585111 
Matthews (MCC): 0.12795285557621297 
TPR: 1.0 
FPR: 0.036539321377117594 
Positive pred val: 0.016992840095465395 
The Matrix (CM): [[178.0, 10297.0], [0.0, 271509.0]] 

P84555 FP
P84556 FP
P36235 FP
P71089 FP
Q91736 FP
Q91571 FP
Q91694 FP
Q91845 FP
Q9PS05 FP
Threshold: 0.01 
ACC: 0.9966948479346346 
Matthews (MCC): 0.3997874562878267 
TPR: 1.0 
FPR: 0.003307239732298106 
Positive pred val: 0.16036036036036036 
The Matrix (CM): [[178.0, 932.0], [0.0, 280874.0]] 

P84555 FP
P84556 FP
P36235 FP
P71089 FP
Q91736 FP
Q91571 FP
Q91694 FP
Q91845 FP
Q9PS05 FP
Threshold: 0.

In [2]:
%%bash
(cat both_r2.out testing_r2_positive.out) > performance_r2.out 
python performance2.py performance_r2.out

P0DJ63 FP
P83605 FP
P85039 FP
P85040 FP
P56409 FP
P0DM47 FP
Q07498 FP
P09759 FP
Q8CBF3 FP
Threshold: 1 
ACC: 0.7215941330004539 
Matthews (MCC): 0.04039804585826999 
TPR: 1.0 
FPR: 0.2785817193388359 
Positive pred val: 0.002262213410604443 
The Matrix (CM): [[178.0, 78506.0], [0.0, 203300.0]] 

P0DJ63 FP
P83605 FP
P85039 FP
P85040 FP
P56409 FP
P0DM47 FP
Q07498 FP
P09759 FP
Q8CBF3 FP
Threshold: 0.1 
ACC: 0.963536938266001 
Matthews (MCC): 0.1280481040845699 
TPR: 1.0 
FPR: 0.036486093269838114 
Positive pred val: 0.017017208413001913 
The Matrix (CM): [[178.0, 10282.0], [0.0, 271524.0]] 

P0DJ63 FP
P83605 FP
P85039 FP
P85040 FP
P56409 FP
P0DM47 FP
Q07498 FP
P09759 FP
Q8CBF3 FP
Threshold: 0.01 
ACC: 0.9967622276441217 
Matthews (MCC): 0.4032672617635113 
TPR: 1.0 
FPR: 0.003239817463077436 
Positive pred val: 0.16315307057745188 
The Matrix (CM): [[178.0, 913.0], [0.0, 280893.0]] 

P0DJ63 FP
P83605 FP
P85039 FP
P85040 FP
P56409 FP
P0DM47 FP
Q07498 FP
P09759 FP
Q8CBF3 FP
Threshold: 0.001